In [1]:
import postgreSQL_functions
import utils
import os
import csv

In [2]:
# conecter au base de donnee Floupics_movies
ma_base_donnees = "Floupics_Movies2"
utilisateur = "postgres"
mot_passe = os.environ.get('pg_psw')

conn = postgreSQL_functions.ouvrir_connection(ma_base_donnees, utilisateur, mot_passe)

type(conn)

psycopg2.extensions.connection

### creation les tables movies, links, ratings, tags dans le base de donnes Floupics_movies



In [3]:
# creation la table movies

sql_creation_table_movies = """
    CREATE TABLE IF NOT EXISTS movies (
    movieId INTEGER primary key,
    title text,
    genres text
    );
"""

postgreSQL_functions.creer_table(conn, sql_creation_table_movies)

La table a été crée avec succès


In [4]:
# creation la table ratings

sql_creation_table_ratings = """
    CREATE TABLE IF NOT EXISTS ratings (
    userId INTEGER NOT NULL,
    movieId INTEGER NOT NULL,
    ratings float4,
    time_stamp VARCHAR,
    PRIMARY KEY (userId, movieId),
    FOREIGN KEY ( movieId) REFERENCES movies(movieId)
    );
"""

postgreSQL_functions.creer_table(conn, sql_creation_table_ratings)

La table a été crée avec succès


In [5]:
# creation la table links

sql_creation_table_links = """
    CREATE TABLE IF NOT EXISTS links (
    movieId INTEGER NOT NULL,
    imdbld INTEGER,
    tmdbld CHAR(13) NULL,
    PRIMARY KEY (imdbld),
    FOREIGN KEY ( movieId) REFERENCES movies(movieId)
    );
"""

postgreSQL_functions.creer_table(conn, sql_creation_table_links)

La table a été crée avec succès


In [6]:
# creation la table tags

sql_creation_table_tags = """
    CREATE TABLE IF NOT EXISTS tags (
    userId INTEGER NOT NULL,
    movieId INTEGER NOT NULL,
    tag VARCHAR NOT NULL,
    time_stamp VARCHAR NOT NULL,
    FOREIGN KEY (movieId) REFERENCES movies(movieId)
    );
"""

postgreSQL_functions.creer_table(conn, sql_creation_table_tags)

La table a été crée avec succès


### import les donnes en CSV par requêtes SQL dans les tables movies, ratings, links, tags

#### inserer les donnees csv dans le table movies

In [19]:
# Chemin vers les différents fichiers de données
chemin_movies =  "data/movies.csv"
chemin_links =  "data/links.csv"
chemin_ratings = "data/ratings.csv"
chemin_tags = "data/tags.csv"

In [20]:
movies_dict = utils.lire_csv_dict(chemin_movies)

In [9]:
sql_inserer_movies_dict = """
    INSERT INTO movies 
    (movieId, title, genres)
    VALUES (%(movieId)s, %(title)s, %(genres)s);
"""

In [10]:
postgreSQL_functions.inserer_donnees(conn, sql_inserer_movies_dict, movies_dict)

Les données ont été insérées avec succès


#### inserer les donnees csv dans le table ratings

In [11]:
ratings_dict = utils.lire_csv_dict(chemin_ratings)

In [12]:
sql_inserer_ratings_dict = """
    INSERT INTO ratings 
    (movieId, userId, ratings, time_stamp)
    VALUES (%(movieId)s, %(userId)s, %(rating)s, %(timestamp)s);
"""

In [13]:
postgreSQL_functions.inserer_donnees(conn, sql_inserer_ratings_dict, ratings_dict)

Les données ont été insérées avec succès


#### Inserer les donnees csv dans la table links

In [14]:
links_dict = utils.lire_csv_dict(chemin_links)

In [15]:
sql_inserer_links_dict = """
    INSERT INTO links 
    (movieId, imdbld, tmdbld)
    VALUES (%(movieId)s, %(imdbId)s, %(tmdbId)s);
"""

In [16]:
postgreSQL_functions.inserer_donnees(conn, sql_inserer_links_dict, links_dict)

Les données ont été insérées avec succès


#### Inserer les donnees csv dans la table tags

In [17]:
tags_dict = utils.lire_csv_dict(chemin_tags)

In [18]:
sql_inserer_tags_dict = """
    INSERT INTO tags 
    (userId, movieId, tag, time_stamp)
    VALUES (%(userId)s, %(movieId)s, %(tag)s, %(timestamp)s);
"""

In [19]:
postgreSQL_functions.inserer_donnees(conn, sql_inserer_tags_dict, tags_dict)

Les données ont été insérées avec succès


### Creation deux nouvelles colonnes, year titre 

In [15]:
sql_creation_colonne_years = """
    ALTER TABLE movies ADD year VARCHAR
"""

sql_creation_colonne_film_title = """
    ALTER TABLE movies ADD film_title VARCHAR
"""

In [8]:
postgreSQL_functions.creer_table(conn, sql_creation_colonne_years) # créer le colonne yrars

La table a été crée avec succès


In [16]:
postgreSQL_functions.creer_table(conn, sql_creation_colonne_film_title) # créer le colonne film_title

La table a été crée avec succès


In [21]:
years = []
film_title = []
title_year = [{}]
for i in range(len(movies_dict)):
    title = movies_dict[i]['title']
    new_title = title.strip()
    year = new_title[-5:-1]
    new_title = new_title[:-6].strip()
    film_title.append(new_title)
    years.append(year)
    


### Insert les donnes dans le colonne years

In [ ]:
sql_inserer_years = """
    INSERT INTO movies 
    (year)
    VALUES (%s);
"""
    
postgreSQL_functions.inserer_donnees(conn, sql_inserer_years, years)

### Insert les donnes dans le colonne film_title

In [ ]:
sql_inserer_film_title = """
    INSERT INTO movies 
    (film_title)
    VALUES (%(film_title)s);
"""
cursor = conn.cursor()
for d in film_title:
    cursor.execute(sql_inserer_film_title, d)
    
    #cursor.execute(sql_inserer_film_title, film_title[d])
    #conn.commit()
#postgreSQL_functions.inserer_donnees(conn, sql_inserer_film_title, film_title)

### Les 15 Films les mieux notés:

In [25]:
postgreSQL_select_Query = """
    SELECT title, avg(ratings)
    FROM movies 
    INNER JOIN ratings on movies.movieId=ratings.movieId
    GROUP BY title
    ORDER BY avg(ratings) DESC 
    LIMIT 15
"""


postgreSQL_functions.lire_donnees(conn, postgreSQL_select_Query)

    

Les données ont été lues avec succès


[('Mystery of the Third Planet, The (Tayna tretey planety) (1981)', 5.0),
 ('Garden of Words, The (Koto no ha no niwa) (2013)', 5.0),
 ('Brother (Brat) (1997)', 5.0),
 ('Into the Woods (1991)', 5.0),
 ('Dream of Light (a.k.a. Quince Tree Sun, The) (Sol del membrillo, El) (1992)',
  5.0),
 ('Meantime (1984)', 5.0),
 ('Nasu: Summer in Andalusia (2003)', 5.0),
 ('Holy Motors (2012)', 5.0),
 ('Watching the Detectives (2007)', 5.0),
 ('Eva (2011)', 5.0),
 ('Story of Women (Affaire de femmes, Une) (1988)', 5.0),
 ('Connections (1978)', 5.0),
 ('Go for Zucker! (Alles auf Zucker!) (2004)', 5.0),
 ('Girls About Town (1931)', 5.0),
 ('What Men Talk About (2010)', 5.0)]

### Les 15 Films les mieux notés ayant reçu plus de 40 notes:

In [26]:
postgreSQL_select_Query = """
    SELECT title, ROUND(avg(ratings)), COUNT(ratings)
    FROM movies 
    INNER JOIN ratings on movies.movieId=ratings.movieId
    GROUP BY title
    HAVING COUNT(ratings) > 40
    ORDER BY avg(ratings) DESC 
    LIMIT 15
"""


postgreSQL_functions.lire_donnees(conn, postgreSQL_select_Query)


Les données ont été lues avec succès


[('Shawshank Redemption, The (1994)', 4.0, 317),
 ('Lawrence of Arabia (1962)', 4.0, 45),
 ('Godfather, The (1972)', 4.0, 192),
 ('Fight Club (1999)', 4.0, 218),
 ('Cool Hand Luke (1967)', 4.0, 57),
 ('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
  4.0,
  97),
 ('Rear Window (1954)', 4.0, 84),
 ('Godfather: Part II, The (1974)', 4.0, 129),
 ('Departed, The (2006)', 4.0, 107),
 ('Goodfellas (1990)', 4.0, 126),
 ('Casablanca (1942)', 4.0, 100),
 ('Dark Knight, The (2008)', 4.0, 149),
 ('Usual Suspects, The (1995)', 4.0, 204),
 ('Princess Bride, The (1987)', 4.0, 142),
 ('Star Wars: Episode IV - A New Hope (1977)', 4.0, 251)]

### Les utilisateurs les plus prolifques en notes

In [27]:
postgreSQL_select_Query = """
    SELECT userId, COUNT(ratings)
    FROM movies 
    INNER JOIN ratings on movies.movieId=ratings.movieId
    GROUP BY userId
    ORDER BY COUNT(ratings) DESC 
    LIMIT 10
"""


postgreSQL_functions.lire_donnees(conn, postgreSQL_select_Query)

Les données ont été lues avec succès


[(414, 2698),
 (599, 2478),
 (474, 2108),
 (448, 1864),
 (274, 1346),
 (610, 1302),
 (68, 1260),
 (380, 1218),
 (606, 1115),
 (288, 1055)]

### Les 10 films recevant le plus de tags

In [28]:
postgreSQL_select_Query = """
    SELECT title, tag, COUNT(tag)
    FROM movies 
    INNER JOIN tags on movies.movieId=tags.movieId
    GROUP BY title, tag
    ORDER BY COUNT(tag) DESC 
    LIMIT 10
"""


postgreSQL_functions.lire_donnees(conn, postgreSQL_select_Query)

Les données ont été lues avec succès


[('Star Wars: Episode IV - A New Hope (1977)', 'sci-fi', 3),
 ('Fight Club (1999)', 'dark comedy', 3),
 ('Inception (2010)', 'thought-provoking', 3),
 ('Donnie Darko (2001)', 'hallucinatory', 3),
 ('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
  'dark comedy',
  3),
 ('Star Wars: Episode IV - A New Hope (1977)', 'classic sci-fi', 3),
 ('Twelve Monkeys (a.k.a. 12 Monkeys) (1995)', 'time travel', 3),
 ('Donnie Darko (2001)', 'dreamlike', 3),
 ('Step Brothers (2008)', 'will ferrell', 3),
 ('Memento (2000)', 'twist ending', 3)]

### Les 25 tags le plus souvent employés, avec ses fréquences

In [29]:
postgreSQL_select_Query = """
    SELECT tag, COUNT(tag)
    FROM tags
    GROUP BY tag
    ORDER BY COUNT(tag) DESC 
    LIMIT 25
"""


postgreSQL_functions.lire_donnees(conn, postgreSQL_select_Query)

Les données ont été lues avec succès


[('In Netflix queue', 131),
 ('atmospheric', 36),
 ('thought-provoking', 24),
 ('superhero', 24),
 ('funny', 23),
 ('surreal', 23),
 ('Disney', 23),
 ('religion', 22),
 ('psychology', 21),
 ('dark comedy', 21),
 ('quirky', 21),
 ('sci-fi', 21),
 ('suspense', 20),
 ('twist ending', 19),
 ('crime', 19),
 ('visually appealing', 19),
 ('politics', 18),
 ('mental illness', 16),
 ('music', 16),
 ('time travel', 16),
 ('comedy', 15),
 ('aliens', 15),
 ('dark', 15),
 ('mindfuck', 14),
 ('dreamlike', 14)]